In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

class SignLanguageDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.labels = self.data.iloc[:, 0].values
        self.images = self.data.iloc[:, 1:].values / 255.0  # Normalize pixel values to [0, 1]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image = self.images[idx].astype(np.float32).reshape(28, 28)  # Reshape to 28x28 image
        label = self.labels[idx]
        return torch.tensor(image), torch.tensor(label)

# Load dataset
train_dataset = SignLanguageDataset('C:\\Users\\DeLL\\OneDrive\\Desktop\\dipali\\minor proj\\Sign language detector1\\sign_mnist_train.csv')
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 14 * 14, num_classes)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 32 * 14 * 14)
        x = self.fc1(x)
        return x

# Initialize the model with the correct number of classes
num_classes = 25  # Update the number of classes
model = SimpleCNN(num_classes)

In [ ]:
import torch.optim as optim

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        
        inputs = inputs.unsqueeze(1).float()  # Add channel dimension (1, 28, 28)
        outputs = model(inputs)
        
        # Calculate loss
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')


In [4]:
label_to_char = {
    0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9',
    10: 'A', 11: 'B', 12: 'C', 13: 'D', 14: 'E', 15: 'F', 16: 'G', 17: 'H', 18: 'I', 19: 'J',
    20: 'K', 21: 'L', 22: 'M', 23: 'N', 24: 'O'
}


In [ ]:
import cv2

def preprocess_image(image):
    # Convert to grayscale and resize to 28x28
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (28, 28))
    normalized = resized / 255.0  # Normalize pixel values to [0, 1]
    return torch.tensor(normalized, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions

# Access webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Preprocess the frame
    input_tensor = preprocess_image(frame)
    
    # Predict the sign
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
        _, predicted = torch.max(output, 1)
        label = predicted.item()
    
    # Map the predicted label to the corresponding character
    sign_character = label_to_char.get(label, "Unknown")
    
    # Display the result
    cv2.putText(frame, f'Predicted Sign: {sign_character}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.imshow('Sign Language Detector', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()